> <h2>EDA</h2>

<h2>import data</h2>

In [72]:
import pandas as pd
df = pd.read_csv('Churn_prediction.csv')

def data_preprocessing(dataset):
    df = dataset.copy()
    columns = ['customerID','gender','SeniorCitizen',
                'Partner','Dependents','tenure','PhoneService',
                'MultipleLines','InternetService','OnlineSecurity',
                'OnlineBackup','DeviceProtection','TechSupport',
                'StreamingTV','StreamingMovies','Contract',
                'PaperlessBilling','PaymentMethod',
                'MonthlyCharges','TotalCharges','Churn']
    # create columns not exist
    notImported=list(set(columns) - set(df.columns))
    if len(notImported):
        print("{0}, column(s) don't(doesn't) exist".format(notImported))
        print('!!! please enter this column first !!!')

    else:
            
        # make TotalCharges numeric                  
        df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
        df['TotalCharges'] = df['TotalCharges'].fillna(0)
        # prepare string variables
        df.columns = df.columns.str.lower().str.replace(' ','_')
        string_columns = list(df.dtypes[df.dtypes=='O'].index)
        for col in string_columns:
            df[col] = df[col].str.lower().str.replace(' ','_')
        # make dependent variable numeric
        df.churn = (df.churn == 'yes').astype(int)
        # drop duplicates
        df.drop_duplicates(inplace=True)
    
        return df

df = pd.read_csv('Churn_prediction.csv')
df = data_preprocessing(df)

<h2>Split dataset to training set and test set</h2>

In [73]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=666)

print("df_train shape: {0}".format(df_train.shape))
print("df_test shape: {0}".format(df_test.shape))

y_train = df_train['churn'].values
y_test = df_test['churn'].values


df_train shape: (5634, 21)
df_test shape: (1409, 21)


<p>Because the percentage of churns and not churns is not the same, we find out accuracy method is not a good choice for model evaluation. So we use other evaluation methods.</p>

In [74]:
# from locale import normalize
df_train.churn.value_counts(normalize=True)

0    0.738374
1    0.261626
Name: churn, dtype: float64

In [75]:
global_mean = df_train.churn.mean()
round(global_mean, 3)

0.262

<h2>Feature importance</h2>

In [76]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
               'phoneservice', 'multiplelines', 'internetservice',
               'onlinesecurity', 'onlinebackup', 'deviceprotection',
               'techsupport', 'streamingtv', 'streamingmovies',
               'contract', 'paperlessbilling', 'paymentmethod']
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [77]:
df_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

<p>Which group of observation has more risk of churn?</p>

In [78]:
for col in categorical:
    df_group = df_train.groupby(by=col).churn.agg(['mean'])
    df_group['diff'] = df_group['mean'] - global_mean
    df_group['risk'] = df_group['mean'] / global_mean
    display(df_group)

,mean,diff,risk
gender,,,
female,0.271028,0.009402,1.035938
male,0.252454,-0.009171,0.964944


,mean,diff,risk
seniorcitizen,,,
0,0.232735,-0.028891,0.889572
1,0.413793,0.152167,1.581622


,mean,diff,risk
partner,,,
no,0.321871,0.060245,1.230271
yes,0.197359,-0.064267,0.754355


,mean,diff,risk
dependents,,,
no,0.307614,0.045988,1.175779
yes,0.154664,-0.106962,0.591163


,mean,diff,risk
phoneservice,,,
no,0.219780,-0.041846,0.840055
yes,0.266116,0.004491,1.017164


...

,mean,diff,risk
streamingtv,,,
no,0.334367,0.072741,1.278034
no_internet_service,0.074380,-0.187246,0.284300
yes,0.290397,0.028771,1.109971


,mean,diff,risk
streamingmovies,,,
no,0.332443,0.070817,1.270682
no_internet_service,0.074380,-0.187246,0.284300
yes,0.292605,0.030979,1.118408


,mean,diff,risk
contract,,,
month-to-month,0.421070,0.159444,1.609435
one_year,0.114933,-0.146693,0.439302
two_year,0.028003,-0.233623,0.107034


,mean,diff,risk
paperlessbilling,,,
no,0.156616,-0.105010,0.598626
yes,0.334335,0.072709,1.277911


,mean,diff,risk
paymentmethod,,,
bank_transfer_(automatic),0.167344,-0.094282,0.639630
credit_card_(automatic),0.151589,-0.110037,0.579412
electronic_check,0.446285,0.184660,1.705815
mailed_check,0.189272,-0.072354,0.723445


<h2>Mutual information</h2>
<p>Mutual information (MI) - concept from information theory , it tells us how much we can learn about one variable if we know the value of another</p>
<a href='https://en.wikipedia.org/wiki/Mutual_information'>https://en.wikipedia.org/wiki/Mutual_information</a>
<p>We find out contract, onlinesecurity, techsupport and internetservice are important independent variables</p>

In [79]:
from sklearn.metrics import mutual_info_score

def calculate_mi(series):
    return mutual_info_score(series, df_train.churn)

df_mi = df_train[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='more_important')

print(df_mi)
categorical_important=['contract', 'onlinesecurity', 'techsupport', 'internetservice']

                  more_important
contract                0.095822
onlinesecurity          0.061679
techsupport             0.060974
internetservice         0.056457
onlinebackup            0.045322
paymentmethod           0.042763
deviceprotection        0.041449
streamingtv             0.030851
streamingmovies         0.030693
paperlessbilling        0.020675
dependents              0.013637
seniorcitizen           0.010546
partner                 0.010131
multiplelines           0.001232
phoneservice            0.000503
gender                  0.000223


<h2>Corrlation between numerical variables and the dependent variable(churn)</h2>
<p>Non of them is important</p>

In [80]:
df_train[numerical].corrwith(df_train.churn).to_frame('correlation')
# But we use them :/
numerical_important = ['tenure', 'monthlycharges', 'totalcharges']

In [81]:
df_train.groupby(by='churn')[numerical].mean()


,tenure,monthlycharges,totalcharges
churn,,,
0,37.338702,61.381190,2539.210577
1,18.225237,74.819064,1557.512246


In [82]:
df_train = df_train[categorical_important+numerical_important+['churn']]
df_train

,contract,onlinesecurity,techsupport,internetservice,tenure,monthlycharges,totalcharges,churn
3905,month-to-month,no,no,dsl,3,35.20,108.95,1
4579,month-to-month,no,no,fiber_optic,31,81.15,2640.55,0
5161,month-to-month,no,yes,dsl,23,54.15,1312.45,0
2168,month-to-month,no,no,fiber_optic,7,74.40,527.90,1
2518,month-to-month,no,no,fiber_optic,24,99.65,2404.85,0
...,...,...,...,...,...,...,...,...
70,month-to-month,no,yes,dsl,1,49.05,49.05,0
2878,month-to-month,no,no,fiber_optic,22,87.00,1850.65,0
1950,month-to-month,no,no,fiber_optic,65,91.85,5940.85,1
1922,month-to-month,no,no,dsl,25,60.35,1404.65,0


In [83]:
[df_train.columns.to_list().index(i) for i in categorical_important]

[0, 1, 2, 3]

In [84]:
df_train

,contract,onlinesecurity,techsupport,internetservice,tenure,monthlycharges,totalcharges,churn
3905,month-to-month,no,no,dsl,3,35.20,108.95,1
4579,month-to-month,no,no,fiber_optic,31,81.15,2640.55,0
5161,month-to-month,no,yes,dsl,23,54.15,1312.45,0
2168,month-to-month,no,no,fiber_optic,7,74.40,527.90,1
2518,month-to-month,no,no,fiber_optic,24,99.65,2404.85,0
...,...,...,...,...,...,...,...,...
70,month-to-month,no,yes,dsl,1,49.05,49.05,0
2878,month-to-month,no,no,fiber_optic,22,87.00,1850.65,0
1950,month-to-month,no,no,fiber_optic,65,91.85,5940.85,1
1922,month-to-month,no,no,dsl,25,60.35,1404.65,0


<h2>One Hot Encoding</h2>

In [86]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
import pickle

OHE = make_column_transformer((OneHotEncoder(), categorical_important ),
                                        remainder='passthrough',
                                        verbose_feature_names_out=False)

ohe = OHE.fit_transform(df_train)
pickle.dump(ohe, open('OneHotEncoder.pkl','wb'))
# ohe = pickle.load(open('OneHotEncoder.pkl','rb'))
df_train = pd.DataFrame(ohe, columns=OHE.get_feature_names_out())

df_train

,contract_month-to-month,contract_one_year,contract_two_year,onlinesecurity_no,onlinesecurity_no_internet_service,onlinesecurity_yes,techsupport_no,techsupport_no_internet_service,techsupport_yes,internetservice_dsl,internetservice_fiber_optic,internetservice_no,tenure,monthlycharges,totalcharges,churn
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0,35.20,108.95,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,31.0,81.15,2640.55,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,23.0,54.15,1312.45,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,7.0,74.40,527.90,1.0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,24.0,99.65,2404.85,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,49.05,49.05,0.0
5630,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,22.0,87.00,1850.65,0.0
5631,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,65.0,91.85,5940.85,1.0
5632,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,25.0,60.35,1404.65,0.0


<h2>Feature scaling</h2>

In [87]:
from sklearn.preprocessing import StandardScaler
import pickle

scaler = StandardScaler()
scaler.fit(df_train)
pickle.dump(scaler, open('scaler.pkl','wb'))

df_train = scaler.transform(df_train)
df_train =pd.DataFrame(df_train, columns=scaler.feature_names_in_)

# scaler = pickle.load(open('scaler.pkl','rb'))


<h2>Creat a function for Feature Engineering</h2>

In [93]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
import pickle

def FeatureEngineering(dataset:pd.DataFrame, categorical_variables:list, numerical_variables:list):
    df = dataset.copy()
    categorical_important = categorical_variables.copy()
    numerical_important = numerical_variables.copy()
    
    df = df[numerical_important+categorical_important]
    # Dummy variables
    OHE = make_column_transformer((OneHotEncoder(), categorical_important ),
                                        remainder='passthrough',
                                        verbose_feature_names_out=False)
    ohe = OHE.fit_transform(df)
    pickle.dump(ohe, open('OneHotEncoder.pkl','wb'))
    df = pd.DataFrame(ohe, columns=OHE.get_feature_names_out())

    # Feature scaling
    scaler = StandardScaler()
    scaler.fit(df)
    pickle.dump(scaler, open('scaler.pkl','wb'))
    df = scaler.transform(df)
    df = pd.DataFrame(df,columns=scaler.feature_names_in_)
    # df.columns = scaler.feature_names_in_
    

    return df

In [97]:
import pandas as pd
df = pd.read_csv('Churn_prediction.csv')

categorical_important = ['contract', 'onlinesecurity', 'techsupport', 'internetservice']
numerical_important = ['tenure', 'monthlycharges', 'totalcharges']

df = data_preprocessing(df)
df = FeatureEngineering(df,
                        categorical_variables=categorical_important,
                        numerical_variables=numerical_important)
df

,contract_month-to-month,contract_one_year,contract_two_year,onlinesecurity_no,onlinesecurity_no_internet_service,onlinesecurity_yes,techsupport_no,techsupport_no_internet_service,techsupport_yes,internetservice_dsl,internetservice_fiber_optic,internetservice_no,tenure,monthlycharges,totalcharges
0,0.904184,-0.514249,-0.562975,1.006696,-0.525927,-0.633933,1.013869,-0.525927,-0.639439,1.381712,-0.885660,-0.525927,-1.277445,-1.160323,-0.992611
1,-1.105970,1.944582,-0.562975,-0.993349,-0.525927,1.577454,1.013869,-0.525927,-0.639439,1.381712,-0.885660,-0.525927,0.066327,-0.259629,-0.172165
2,0.904184,-0.514249,-0.562975,-0.993349,-0.525927,1.577454,1.013869,-0.525927,-0.639439,1.381712,-0.885660,-0.525927,-1.236724,-0.362660,-0.958066
3,-1.105970,1.944582,-0.562975,-0.993349,-0.525927,1.577454,-0.986321,-0.525927,1.563872,1.381712,-0.885660,-0.525927,0.514251,-0.746535,-0.193672
4,0.904184,-0.514249,-0.562975,1.006696,-0.525927,-0.633933,1.013869,-0.525927,-0.639439,-0.723740,1.129102,-0.525927,-1.236724,0.197365,-0.938874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,-1.105970,1.944582,-0.562975,-0.993349,-0.525927,1.577454,-0.986321,-0.525927,1.563872,1.381712,-0.885660,-0.525927,-0.340876,0.665992,-0.127605
7039,-1.105970,1.944582,-0.562975,1.006696,-0.525927,-0.633933,1.013869,-0.525927,-0.639439,-0.723740,1.129102,-0.525927,1.613701,1.277533,2.242606
7040,0.904184,-0.514249,-0.562975,-0.993349,-0.525927,1.577454,1.013869,-0.525927,-0.639439,1.381712,-0.885660,-0.525927,-0.870241,-1.168632,-0.852932
7041,0.904184,-0.514249,-0.562975,1.006696,-0.525927,-0.633933,1.013869,-0.525927,-0.639439,-0.723740,1.129102,-0.525927,-1.155283,0.320338,-0.870513
